In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,29118
2,Huelva,Confirmados PDIA 14 días,990
3,Huelva,Tasa PDIA 14 días,"192,9185260245143"
4,Huelva,Confirmados PDIA 7 días,495
5,Huelva,Total Confirmados,29294
6,Huelva,Curados,25745
7,Huelva,Fallecidos,370


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  29118.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  7594.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 861 personas en los últimos 7 días 

Un positivo PCR cada 494 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,29118.0,495.0,990.0,513170.0,96.459263,192.918526,101.0
Huelva-Costa,16945.0,254.0,457.0,289548.0,87.722934,157.832207,61.0
Huelva (capital),7594.0,167.0,291.0,143837.0,116.103645,202.312340,39.0
Condado-Campiña,9247.0,205.0,457.0,156231.0,131.215956,292.515570,34.0
Punta Umbría,728.0,15.0,25.0,15355.0,97.688049,162.813416,10.0
Sierra de Huelva-Andévalo Central,2584.0,34.0,67.0,67391.0,50.451841,99.419804,8.0
Almonte,1604.0,69.0,149.0,24507.0,281.552210,607.989554,8.0
Bollullos Par del Condado,868.0,43.0,85.0,14387.0,298.880934,590.811149,7.0
Valverde del Camino,487.0,12.0,21.0,12750.0,94.117647,164.705882,5.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cortelazor,8.0,1.0,3.0,299.0,334.448161,1003.344482,0.0
Almonte,1604.0,69.0,149.0,24507.0,281.552210,607.989554,8.0
Bollullos Par del Condado,868.0,43.0,85.0,14387.0,298.880934,590.811149,7.0
Galaroza,35.0,8.0,8.0,1382.0,578.871201,578.871201,3.0
Rociana del Condado,527.0,12.0,29.0,7939.0,151.152538,365.285300,1.0
Condado-Campiña,9247.0,205.0,457.0,156231.0,131.215956,292.515570,34.0
Cumbres de San Bartolomé,11.0,1.0,1.0,376.0,265.957447,265.957447,0.0
Bonares,259.0,4.0,16.0,6060.0,66.006601,264.026403,2.0
Moguer,1121.0,16.0,49.0,21867.0,73.169616,224.081950,3.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Aracena,513.0,1.0,6.0,8255.0,12.113870,72.683222,0.0,0.166667
Cortegana,200.0,1.0,5.0,4602.0,21.729683,108.648414,0.0,0.200000
Bonares,259.0,4.0,16.0,6060.0,66.006601,264.026403,2.0,0.250000
Moguer,1121.0,16.0,49.0,21867.0,73.169616,224.081950,3.0,0.326531
Cortelazor,8.0,1.0,3.0,299.0,334.448161,1003.344482,0.0,0.333333
Rosal de la Frontera,29.0,1.0,3.0,1697.0,58.927519,176.782557,0.0,0.333333
Beas,202.0,3.0,8.0,4341.0,69.108500,184.289334,0.0,0.375000
Ayamonte,1296.0,10.0,26.0,21104.0,47.384382,123.199393,1.0,0.384615
San Juan del Puerto,457.0,7.0,18.0,9411.0,74.381043,191.265540,2.0,0.388889
